In [ ]:
pip install chembl_webresource_client

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client
import os
import numpy as np

In [ ]:
target = new_client.target
target_query = target.search('PARP1')
targets = pd.DataFrame.from_dict(target_query)
targets

In [ ]:
selected_target = targets.target_chembl_id[0]
selected_target

In [ ]:
ac = new_client.activity
raw = ac.filter(target_chembl_id=selected_target)
df0 = pd.DataFrame.from_dict(raw)
df0

In [ ]:
df0.to_csv('PARP1_00_bioactivity_data_raw.csv', index=False)

In [ ]:
res_IC50 = ac.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df_IC50 = pd.DataFrame.from_dict(res_IC50)

In [ ]:
df_IC50.to_csv('PARP1_01_bioactivity_IC50_data_raw.csv', index=False)

In [ ]:
#Remove missing data
df2_IC50 = df_IC50[df_IC50.standard_value.notna()]
df2_IC50 = df2_IC50[df_IC50.canonical_smiles.notna()]

In [ ]:
#Remove duplicate data in canonical_smiles column
df2_nr_IC50 = df2_IC50.drop_duplicates(['canonical_smiles'])

In [ ]:
#Selecting 3 column for further analysis
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3_IC50 = df2_nr_IC50[selection]

In [ ]:
df3_IC50.to_csv('PARP1_02_bioactivity_IC50_data_preprocessed.csv', index=False)

In [ ]:
df4_IC50 = pd.read_csv('PARP1_02_bioactivity_IC50_data_preprocessed.csv')

In [ ]:
bioactivity_threshold = []
for i in df4_IC50.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")
    
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5_IC50 = pd.concat([df4_IC50, bioactivity_class], axis=1)

In [ ]:
df5_IC50.to_csv('PARP1_03_bioactivity_IC50_data_curated.csv', index=False)